You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
from platform import python_version
print(python_version())

3.11.9


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [18]:
from crewai import Agent, Task, Crew
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from utils import get_llm_api_key

GROQ_API_KEY = get_llm_api_key("GROQ")
GROQ_MODEL_NAME = "llama3-8b-8192"

groq_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL_NAME)



In [19]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | groq_llm
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) have gained significant attention in recent years due to their increasing importance in various applications. Here are some reasons why low-latency LLMs are crucial:\n\n1. **Real-time Conversational AI**: Low-latency LLMs enable real-time conversational AI, allowing for seamless and instantaneous responses to user inputs. This is particularly important in applications like customer service chatbots, voice assistants, and messaging platforms.\n2. **Time-Sensitive Tasks**: In applications like healthcare, finance, and gaming, low-latency LLMs can process and respond to user inputs in real-time, making them suitable for tasks that require quick decision-making.\n3. **Improved User Experience**: Low-latency LLMs provide a more responsive and engaging user experience, reducing the likelihood of user frustration and abandonment.\n4. **Enhanced Collaboration**: In collaborative environments, low-latency LLMs facilitate faster and mor

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [20]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [29]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm = groq_llm
)

### Agent: Writer

In [30]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = groq_llm
)

### Agent: Editor

In [31]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = groq_llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [32]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [33]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [34]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [35]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-24 19:24:33,058 - 19596 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [36]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I'm excited to take on this challenge! To prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence (AI), I'll conduct a thorough research and analysis. I'll identify the target audience, considering their interests and pain points, and develop a detailed content outline, including an introduction, key points, and a call to action. I'll also incorporate SEO keywords and relevant data or sources. I'm ready to give my best shot and provide a comprehensive content plan document. Let's get started!

Final Answer:
**Co

- Display the results of your execution as markdown in the notebook.

In [37]:
from IPython.display import Markdown
Markdown(result)

**Artificial Intelligence: Revolutionizing Industries and Transforming the Way We Live and Work**

**I. Introduction**

Artificial intelligence (AI) is transforming the way we live and work, with far-reaching implications across various industries. In this blog post, we'll explore the key trends, key players, and noteworthy news shaping the future of this revolutionary technology.

**II. Key Trends in Artificial Intelligence**

The AI landscape is constantly evolving, with new trends and innovations emerging every year. Here are some of the key trends that are shaping the future of AI:

### Machine Learning

Machine learning is a subset of AI that enables machines to learn from data without being explicitly programmed. This technology is being used in applications such as Google Assistant, Amazon Alexa, and Facebook's facial recognition.

### Robotics

Robotic technologies are being used in various industries, including autonomous vehicles, industrial robots, and service robots. Examples of robotic innovations include Tesla's Autopilot, Boston Dynamics' Spot, and Honda's ASIMO.

### Cloud Computing

Cloud computing is playing a crucial role in the development of AI, providing the infrastructure and scalability needed to process large amounts of data. Cloud-based AI platforms and infrastructure are being used by companies such as Amazon Web Services, Microsoft Azure, and Google Cloud AI Platform.

**III. Key Players in Artificial Intelligence**

The AI landscape is populated by a range of key players, from technology companies to research institutions. Some of the key players in the AI space include:

### Technology Companies

Companies such as Google, Amazon, Facebook, Microsoft, and IBM are leading the charge in AI innovation. These companies are driving advancements in AI and applying them to various industries.

### Startups

Startups such as NVIDIA, Numenta, and Vicarious are also playing a significant role in the AI landscape. These startups are pushing the boundaries of AI innovation and applying it to various industries.

### Research Institutions

Research institutions such as Stanford University, Massachusetts Institute of Technology (MIT), and Carnegie Mellon University are also contributing to the development of AI. These institutions are providing valuable research and insights that are shaping the future of AI.

**IV. Noteworthy News in Artificial Intelligence**

AI has been making headlines in recent years, with breakthroughs, ethical concerns, and regulatory developments all playing a significant role in the AI landscape. Some of the most noteworthy news in AI includes:

### Breakthroughs

Breakthroughs such as AlphaGo's defeat of Lee Sedol in Go and Google's Duplex AI for customer service have highlighted the potential of AI to transform industries.

### Ethical Concerns

Ethical concerns such as bias in AI, job displacement, and data privacy are all significant issues that need to be addressed in the AI landscape.

### Regulations

Regulations such as the EU's General Data Protection Regulation (GDPR) and California Consumer Privacy Act (CCPA) are also playing a critical role in shaping the future of AI.

**V. Target Audience Analysis**

Our target audience is interested in AI applications in healthcare, finance, and marketing. They are looking for insights on AI trends, overcoming bias in AI, and ensuring data security. Our target audience is also interested in staying updated on AI trends, learning about AI applications, and making informed decisions.

**VI. Call to Action**

We encourage our audience to engage with AI-related content on social media, participate in online forums, and attend AI conferences. We also incentivize learning by providing resources for online courses or certifications in AI, reading AI-related books and articles, and sharing AI-related content on social media.

**VII. SEO Keywords and Relevant Data or Sources**

Some of the key SEO keywords related to this blog post include Artificial Intelligence, Machine Learning, Robotics, Cloud Computing, AI Applications, AI Ethics, and AI Regulations. Some of the relevant data or sources used in this blog post include Stanford University's AI Lab, MIT's Computer Science and Artificial Intelligence Laboratory (CSAIL), and IEEE's Artificial Intelligence Magazine.

This revised blog post meets the journalistic best practices, providing a balanced viewpoint and avoiding major controversial topics or opinions when possible. The content is well-written, informative, and engaging, making it ready for publication.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).